#practical implementation of a Multi-Agent System using LangGraph with: Network Pattern and Supervisor Pattern

#🔧 Step 1: Install Required Libraries

In [14]:
# Install the required libraries in Colab (quietly suppressing extra output)
!pip install --quiet langchain langgraph google-generativeai


#🔐 Step 2: Set Environment Variable for Gemini API Key

In [15]:
import os
from getpass import getpass

# Prompt user to enter their free Google Gemini API key securely
# Get the key from: https://makersuite.google.com/app/apikey
os.environ["GOOGLE_API_KEY"] = getpass("🔑 Enter your Gemini API Key: ")


🔑 Enter your Gemini API Key: ··········


#🧠 Step 3: Configure Gemini LLM for LangChain

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

# Initialize Gemini LLM with model "gemini-1.5-flash"
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=os.environ["GOOGLE_API_KEY"],
    convert_system_message_to_human=True
)

# Test if Gemini API is working
llm.invoke([HumanMessage(content="Hello, are you working?")])


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


AIMessage(content="Yes, I am working.  I'm ready to assist you with your requests.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--7f684e1e-1b7f-43e4-8b0d-925938cc269c-0')

#🧭 Step 4: Create Agent Functions for Multi-Agent System

In [17]:
from langchain_core.runnables import Runnable
from langgraph.graph import END, StateGraph
from typing import TypedDict

# Define the structure of the shared agent state
class AgentState(TypedDict):
    input: str
    summary: str
    agent_1_response: str
    agent_2_response: str

# Agent 1: Extracts key points from the input
def agent1(state: AgentState):
    user_input = state['input']
    response = llm.invoke(f"Extract key points from: {user_input}")
    return {"agent_1_response": response.content}

# Agent 2: Suggests improvements to the input text
def agent2(state: AgentState):
    user_input = state['input']
    response = llm.invoke(f"Suggest improvements for the text: {user_input}")
    return {"agent_2_response": response.content}

# Supervisor: Combines outputs of agent1 and agent2 into a final summary
def supervisor(state: AgentState):
    combined = f"Summary:\n{state['agent_1_response']}\n\nSuggestions:\n{state['agent_2_response']}"
    return {"summary": combined}


#🔗 Step 5: Define LangGraph Network Pattern (Parallel Agents)

In [18]:
# Create a LangGraph state machine for parallel agent execution
network_graph = StateGraph(AgentState)

# Add all agent and supervisor nodes
network_graph.add_node("agent1", agent1)
network_graph.add_node("agent2", agent2)
network_graph.add_node("supervisor", supervisor)

# Connect agent1 → supervisor and agent2 → supervisor
network_graph.add_edge("agent1", "supervisor")
network_graph.add_edge("agent2", "supervisor")

# Both agents start execution from the entry point
network_graph.set_entry_point("agent1")
network_graph.set_entry_point("agent2")

# Supervisor is the final node
network_graph.set_finish_point("supervisor")

# Compile the graph
network_app = network_graph.compile()


#🧪 Step 6: Test Network Pattern (Parallel Execution)

In [19]:
# Sample input text
input_text = "OpenAI's ChatGPT has revolutionized conversational AI by introducing context-aware large language models into mainstream applications."

# Invoke the parallel agent system
result = network_app.invoke({"input": input_text})

# Print final output from supervisor
print("📋 Final Output (Network Pattern):\n")
print(result["summary"])


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


📋 Final Output (Network Pattern):

Summary:
* **ChatGPT revolutionized conversational AI.**  This highlights its significant impact.
* **Context-aware large language models.** This explains the technology behind the revolution.
* **Mainstream applications.** This points to its widespread adoption and accessibility.

Suggestions:
Here are a few suggestions for improvement, each with a slightly different emphasis:

**Option 1 (More concise):**

> ChatGPT, from OpenAI, revolutionized conversational AI by bringing context-aware large language models to mainstream applications.


**Option 2 (Emphasis on impact):**

> OpenAI's ChatGPT has dramatically reshaped the conversational AI landscape by introducing context-aware large language models to mainstream use.


**Option 3 (More specific):**

> OpenAI's ChatGPT has revolutionized conversational AI by making context-aware large language models accessible and widely adopted in mainstream applications.


**Option 4 (More cautious, acknowledging

#👮‍♂️ Step 7: Define Supervisor Pattern (Sequential Execution)

In [20]:
# New state graph for sequential flow: agent1 → agent2 → supervisor
supervisor_graph = StateGraph(AgentState)

# Add same nodes
supervisor_graph.add_node("agent1", agent1)
supervisor_graph.add_node("agent2", agent2)
supervisor_graph.add_node("supervisor", supervisor)

# Define sequential path
supervisor_graph.set_entry_point("agent1")            # Start with agent1
supervisor_graph.add_edge("agent1", "agent2")          # Then go to agent2
supervisor_graph.add_edge("agent2", "supervisor")      # Finally go to supervisor

# End node
supervisor_graph.set_finish_point("supervisor")

# Compile supervisor pattern
supervisor_app = supervisor_graph.compile()


#🧪 Step 8: Test Supervisor Pattern (Sequential Execution)

In [21]:
# Run same input through supervisor-controlled flow
result2 = supervisor_app.invoke({"input": input_text})

# Print result
print("🧠 Final Output (Supervisor Pattern):\n")
print(result2["summary"])


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


🧠 Final Output (Supervisor Pattern):

Summary:
* **ChatGPT revolutionized conversational AI:**  It significantly advanced the field.
* **Context-aware large language models:**  Its core technology is based on these models, allowing for more natural and coherent conversations.
* **Mainstream applications:**  ChatGPT's impact extends beyond research and is now used in widely accessible applications.

Suggestions:
Here are a few suggestions for improving the text, each with a slightly different emphasis:

**Option 1 (More concise):**

> ChatGPT, from OpenAI, has revolutionized conversational AI by bringing context-aware large language models to mainstream applications.

This version is shorter and more direct, prioritizing clarity.

**Option 2 (More impactful):**

> OpenAI's ChatGPT has revolutionized conversational AI, bringing the power of context-aware large language models to mainstream applications for the first time.

This version emphasizes the novelty of ChatGPT's achievement.

**

#✅ Output Behavior

| Agent        | Task                  | Model            |
| ------------ | --------------------- | ---------------- |
| `agent1`     | Extracts key points   | Gemini 1.5 Flash |
| `agent2`     | Suggests improvements | Gemini 1.5 Flash |
| `supervisor` | Combines & summarizes | Final Step       |
